# Projekt 5 - Infravörös távolság mérés plottolással

Az előző projekten keresztül szert tettünk egy klasszra, amivel egy aktív infravörös szenzor objektumot hozhatunk létre. Mint említettük ennek egyik előnye a könnyebb bővíthetőség és módosítása a viselkedési formának. 

Ezek után kézenfekvőnek tűnik, hogy a mért távolságot ne csak a képernyőre írjuk ki, hanem ábrázoljuk is grafikonon amit folytonosan frsissítünk. Így kaphatunk egy grafikus visszajelzést arról, hogyan változott egy adott tárgy távolsága a szenzorunktól. 

A jelen projektben a már meglévő szenzor klasszunkat kibővítjük, hogy egy parancsszóra elindítsa a mért távolságok ábrázolását, illetve egy másik parancsszóra leállítsa azt. Mindezt egy *thread*ből indítjuk el.

## Mit fogsz készíteni?

Az elrendezésünk elsősorban egy IR forrásból (LED) és egy szenzorból fog állni, a hozzájuk tartozó ellenállásokkal együtt. A jel digitalizálására egy MCP3008-as ADC-t használunk. Másodsorban bemutatjuk, hogyan lehet a mért jelet felerősíteni a volt tartományba egy LM358-as operációs erősítő és a hozzá tartozó ellenállások segítségével. 

## Mit tanulsz meg?

Az infravörös szenzor elkészítésével a következőket tanulod meg:

* Hogyan működik az IR szenzor, hogyan digitalizáljuk az analóg jelet és olvassuk ki azt Pythonból.
* Hogyan inicializáljunk egy IR LED-et és MCP3008 ADC-t.
* Hogyan kapcsoljuk ki- és be az IR LED-et és hogyan olvassuk ki a szenzor feszültségét az ADC-n keresztül.
* Hogyan olvassunk be fájlokat és nyerjük ki belőle az információt.
* Hogyan kell interpolálni.
* Mi az az ```event```?
* Hogyan kell függvényeket elindítani a ```threading``` segítségével.
* Hogyan kell klasszokat létrehozni.
* Hogyan használjuk a ```matplotlib``` csomagot ábrázolásra.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Beimportálni és inicializálni a ```LED``` és ```MCP3008``` objektumokat.
* Definiálni a távolság mérést végző klasszt.
* A klasszban megírni a távolság méréshez tartozó függvényeket.
* Megírni a mért távolság folytonos ábrázolásához tartozó függvényeket.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) IR LED, 940 nm: [itt vásárolhatsz](https://www.tme.eu/hu/details/lte-4206/ir-led-ek/liteon/)

c) IR szenzor, 940 nm: [itt vásárolhatsz](https://www.tme.eu/hu/details/bpv10nf/fotodiodak/vishay/)

d) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

e) Ellenállás: [itt vásárolhatsz](https://www.tme.eu/hu/katalog/metal-film-tht-ellenallasok_112313/?s_order=desc&search=ellenallas&s_field=1000011)

f) [MCP3008 ADC](https://www.tme.eu/hu/details/mcp3008-i_p/a-d-konverterek-ic-k/microchip-technology/)

g) [LM358 operációs erősítő](https://www.tme.eu/hu/details/lm358n_nopb/tht-muveleti-erositok/texas-instruments/)

## A kapcsolási rajz

<img src="bevezeto/prog01_schema.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t.

1) Kössük össze a Raspberry Pi egyik földelését az MCP3008 AGND ás DGND lábaival (fekete drót).

2) Kössük az MCP3008 *VDD* és *VREF* nevű lábait a Raspberry Pi 3.3 V-os tüskéjére. 

3) Kössük az MCP3008 *CLK* nevű lábát a Raspberry Pi *GPIO11* tüskéjére. 

4) Kössük az MCP3008 *DOUT* nevű lábát a Raspberry Pi *GPIO09* tüskéjére. 

5) Kössük az MCP3008 *DIN* nevű lábát a Raspberry Pi *GPIO10* tüskéjére.

6) Kössük az MCP3008 *CS* nevű lábát a Raspberry Pi *GPIO08* tüskéjére.

7) Az LM358 erősítő 8-as lábát kössük a Raspberry Pi 3.3 V-os tüskéjére.

8) Az LM358 erősítő 4-es és 3-as (nem invertáló) lábát kössük a földelésre (GND) tüskéjére.

9) A fotodióda anódját (pozitív, hosszabb láb) kössük szintén a földelésre.

10) A fotodióda katódját (negatív, rövidebb láb) kössük az LM358 erősítő 2-es (invertáló) lábára.

11) Az LM358 erősítő 2-es (invertáló) és 1-es (kimenet) lába közéiktassunk be egy R1 = 1 MOhmos ellenállást.

12) Az LM358 erősítő 1-es (kimenet) lábát kössük át a másik oldalon elhelyezkedő erősítő sorának 5-ös (nem invertáló) lábára.

13) Az LM358 erősítő 6-os lába (invertáló) és a földelés (GND) közé kössünk be az R2 = 1 kOhm ellenállást.

14) Az LM358 erősítő 6-os lába (invertáló) és a 7-es (kimenet) lába közé kössünk be az R3 = 10 kOhm ellenállást. Az R2 és R3 ellenállások ilyen választásával 11-szeres erősítést érhetünk el a jelen.

15) Az IR LED anódját (pozitív, hosszabb láb) kössük a Raspberry Pi *GPIO02*-es tüskéjére, míg a katódját (negatív, rövidebb láb) kössük sorba egy 200 Ohmos ellenállással.

16) A 200 Ohmos ellenállás másik lábát kössük a földelésre (GND).

## A kód

Nyissunk meg egy új python fájlt és mentsük el pl. ```ir_classplotting.py``` név alatt. A ```gpiozero``` csomagnak nincs beépített objektuma ami általánosan az IR szenzort tudná kezelni, így mi fogjuk megoldani a szenzorral való kommunikációt.

Az előző projektekben már tárgyaltuk, hogyan tudunk létrehozni egy klasszt ami elindít és leállít egy távolságmérést. Felhasználva az ott felgyülemlett információt, onnan folytatjuk tovább az ábrázolás elvégzéséhez is.

Érdemes a fontosabb kódolások előtt letesztelni, hogy tudjuk-e szoftveresen vezérelni a LED-et illetve az MCP3008 ADC-t. Ezek mikéntjére itt nem térünk ki, az előző projektben ezt részleteztük.

### Segédfüggvények beimportálása és eddigi kódok

A lenti kódrészletben megnézhetjük, hogy milyen csomagokat importáltunk eddig be és hogy milyen tulajdonságokkal bír az eddig megírt klasszunk. Frissítésként leírjuk, hogy az ```ActiveSensor``` klasszunkat egy IR LED objektummal, egy ADC konverterrel és egy kalibrációs fájlnévvel kell inicializálnunk. Ezen felül van egy ```start``` és ```stop``` metódusa, amik a mérés elindításáért és leállításáért felelős viselkedések. Található még egy ```initialize_calibration``` metódus, ami elvégzi a mért feszültség távolsággá alakítását, és egy ```start_measurement``` metódus, amiben maga a mérés zajlik.

```ir_classplotting.py```:

In [51]:
import threading
import time
from raspberry_functions import read_2column_files, interpolate1d
from gpiozero import LED, MCP3008


class ActiveSensor:
	def __init__(self, led, mcp, calibname):
		self.led = led
		self.mcp = mcp
		self.calibfile = calibname
		self.initialize_calibration(self.calibfile)
		self.event = threading.Event()
		
	def start(self):
		self.event.clear()
		self.led.on()
		t = threading.Thread(target=self.start_measurement)
		t.start()

	def start_measurement(self):
		print('Measurement started')
		while not self.event.is_set():
			current_voltage = self.mcp.voltage
			current_distance = interpolate1d(self.calib_volt, self.calib_distance, current_voltage)
			print(f'Current distance from object is: {current_distance:.2} cm')
			time.sleep(1)
		
	def stop(self):
		self.event.set()
		self.led.off()
		print('Measurement is stopped')

	def initialize_calibration(self, filename):
		self.calib_volt, self.calib_distance = read_2column_files(filename, header=True)
	

if __name__ == '__main__':
	ir = LED(2)
	mcp = MCP3008(channel=7)
	calib_file = 'ir_calibration.csv'
	a = ActiveSensor(ir, mcp, calib_file)


### A hiányzó metódusok összegzése

A mért adatok ábrázolásához nagy vonalakban a következő lépésekre lesz szükség:

* Rögzíteni a mérés időpontját.
* Összegyűjteni listákba az adatokat amit ábrázolni akarunk, ```prepare_data``` metódus.
* Inicializálni egy ábrát, amire a méréseket ábrázoljuk, ```plot_initialize``` metódus.
* Ábrázolás indítása, ```start_plot``` és ```plot_distance_thread``` metódus.
* Ábrázolás leállítása, ```stop_plot``` metódus.

A fenti pontokon kívül szükségünk lesz még néhány segédcsomagra, amit be kell importálnunk. Ezekből is a legfontosabb az a kód első két sora. Be kell állítani egy olyan GUI kezelőt, ami képes a *thread*ek kezelésére. Mi itt a *PyQT5*ből származó ```QtA5Agg``` motort fogjuk használni. A legtöbb esetben nem ez a motor az alap beállítás, úgyhogy a biztonság kedvéért a kód első két sorában ez kell szerepeljen: ```import matplotlib```, ```matplotlib.use('Qt5Agg')```.

Az eddig felsorolt csomagok mellé beimportáljuk még a ```datetime``` csomagot, hogy a mérési időpontokat tudjuk rögzíteni, illetve a ```matplotlib.pyplot``` csomagot, hogy ábrázolni tudjunk.

Ugyanevvel a lendülettel, egy apró módosítást végzünk a már előzőleg megírt ```start_measurement``` metóduson. Ebben a metódusban végezzük a feszültségmérést, így kézenfekvő, hogy itt mérjük meg a mérés időpontját is, ```dd = dt.datetime.now()```. Miután megtörténik a feszültség kalibrálása távolsággá, előkészítjük a mért adatokat az ábrázolásra, ```self.prepare_data(dd, current_distance)```. Ennek a függvénynek nincs visszaadott paramétere, ugyanis a ```self``` objektumhoz rendeljük hozzá az új tulajodnságokat, a ```self.dlist``` időpont listát és a ```self.ylist``` távolság listát (lásd később). Ezeket a listákat az ```__init__``` függvényben inicializáltuk üres listára, így, amikor az objektum inicializálódik, ezek az üres listák is létrejönnek.

Az ```__init__``` függvényben van még egy új elem, a ```self.event_plot = threading.Event()```. Ez azt jelenti, hogy a plottoláshoz is rendelünk egy eseményt, aminek a ki-be kapcsolásával elindíthatjuk és leállíthatjuk az ábrázolást.

In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')
import threading
import time
from raspberry_functions import read_2column_files, interpolate1d
from gpiozero import LED, MCP3008
import datetime as dt
import matplotlib.pyplot as plt


class ActiveSensor:
	def __init__(self, led, mcp, calibname):
		self.led = led
		self.mcp = mcp
		self.calibfile = calibname
		self.initialize_calibration(self.calibfile)
		self.event = threading.Event()
		self.event_plot = threading.Event()   # plottolasi esemeny
		self.dlist = []  # idopont tarolo ures lista
		self.ylist = []  # meresi erteket tarolo ures lista
		
	def start(self):
		self.event.clear()
		self.led.on()

	def start_measurement(self):
		print('Measurement started')
		while not self.event.is_set():
			dd = dt.datetime.now()  # idopont meghatarozasa
			current_voltage = self.mcp.voltage
			current_distance = interpolate1d(self.calib_volt, self.calib_distance, current_voltage)
			print(f'Current distance from object is: {current_distance:.2} cm')
			self.prepare_data(dd, current_distance)  # abrazolashoz az adatok elokeszitese
			time.sleep(1)
		
	def stop(self):
		self.event.set()
		self.led.off()
		print('Measurement is stopped')

	def initialize_calibration(self, filename):
		self.calib_volt, self.calib_distance = read_2column_files(filename, header=True)

	def prepare_data(self, dd, yy, maxlen=20):
		pass
    
	def plot_initialize(self):
		pass

	def start_plot(self):
		pass

	def stop_plot(self):
		pass

	def plot_distance_thread(self):
		pass


if __name__ == '__main__':
	ir = LED(2)
	mcp = MCP3008(channel=7)
	calib_file = 'ir_calibration.csv'
	a = ActiveSensor(ir, mcp, calib_file)


#### Az ábrázolni kívánt adatok előkészítése

Mint ahogy az előbb már láthattuk, a ```prepare_data``` metódus felel azért, hogy az ábrázolandó adatok megfelelő formába kerüljenek. Négy bemenő paramétere van:

* ```self``` - ez maga az objektum, így tudunk hivatkozni a függvényen belül az objektum tulajdonságaira.
* ```dd```   - a legutóbbi méréshez tartozó dátum ```datetime``` formátumban.
* ```yy```   - a legutóbbi mérés értéke.
* ```maxlen``` - ez a paraméter adja meg, hogy a legutolsó hány mérést ábrázoljuk. Alapértelmezettként a legutolsó 20 mérés van megadva.

A függvény első dolga, hogy a paraméterként megadott új mérési időt és mérési értéket hozzáaddja ezen paraméterek tárolólistáihoz, ```self.dlist``` és a ```self.ylist```. A következő lépésben pedig megnézzük, hogy az így kapott időtároló lista hosszabb-e az előre megszabott értéktől (ami alapértelmezetten 20 mérést takar), ```if len(self.dlist) > maxlen:```. Ha túlléptünk a megengedett hosszon, akkor mind a két listából dobjuk ki a legelső (azaz a legrégebben bekerült) elemet a ```pop(0)``` metódussal. Így biztosítjuk, hogy mindig a maximálisan megengedett mennyiségű mérési adatot ábrázoljuk. 

In [ ]:
def prepare_data(self, dd, yy, maxlen=20):
		self.dlist.append(dd)
		self.ylist.append(yy)
		if len(self.dlist) > maxlen:
			self.dlist.pop(0)
			self.ylist.pop(0)

#### Az ábra inicializálása

Ezután létrehozunk, egy metódust, ```plot_initialize``` aminek meghívásával inicializlunk egy grafikont. Bemenő paraméterként csak a ```self``` objektumot adjuk meg, hogy lássa azt. Első dolgunk a plottolás interaktívvá tétele (azaz, hogy plottolás után visszanyerjük a kurzort a terminálban), ```plt.ion()```. Definiálunk egy ábra panelt, ```self.figure``` és a hozzátartozó tengelyeket ```self.ax```, ```self.figure, self.ax = plt.subplots(figsize=(8,6))```. Paraméterként azt is megadjuk, mekkora legyen az ábra, ```figsize=(8,6)```. A következő sorban ábrázoljuk a ```self.dlist```-ben és ```self.ylist```-ben tárolt értékeket, vonallal összekötött pontokként, ```'o-'```. Az így kapott vonal objektumot hozzárendeljük a ```self.line1, = self.ax.plot(self.dlist, self.ylist, 'o-')```, változóhoz. 

Stilizálásként adunk címet az ábrának a ```plt.title``` függvénnyel, beállítjuk az *x* és *y* tengely megnevezését, ```plt.xlabel("Time",fontsize=18)``` és ```plt.ylabel("Distance (cm)",fontsize=18)```. Végül teszünk négyzethálót az ábrára, ```plt.grid(True)```, hogy könnyebben leolvashassuk a mért értékeket.

Ez a metódus önmagától nem fut le. Mielőtt szeretnénk elindítani az ábrázolást, akkor kell meghívnunk ezt a metódust.

In [1]:
def plot_initialize(self):
	plt.ion()
	self.figure, self.ax = plt.subplots(figsize=(8,6))
	self.line1, = self.ax.plot(self.dlist, self.ylist, 'o-')
	plt.title("Dynamic Plot of measurement",fontsize=25)
	plt.xlabel("Time",fontsize=18)
	plt.ylabel("Distance (cm)",fontsize=18)
	plt.grid(True)

#### Az ábrázolás elindítása és leállítása

A következő két metódus a mérés indítása, ```start_plot```,  és leállítása, ```stop_plot```. 

A ```start_plot``` metódusnak egy bemenő paramétere van, a ```self```. Rajta keresztül elérhetővé válik az ábrázolási esemény, ```self.event_plot```. A mérés elindításakor a legelső lépés az az, hogy ezt az eseményt hamis állapotba tedjuk, jelezve, hogy nincs leállítási esemény, ```self.event_plot.clear()```. Ezután létrehozunk egy *thread*et, ```t = threading.Thread(target=self.plot_distance_thread)```, aminek a feladata a ```self.plot_distance_thread``` függvény elindítása lesz, ami a plottolásért felel. Végül a elindítjuk a *thread*et, ```t.start()```.

A plottolás leállításához a bemenő paraméter szintén a ```self```. Így itt, átállíthatjuk a plottolás esemény állapotát igazra, ```self.event_plot.set()``` jelezve, hogy bekövetkezett a leállítás kérése. Plusz kiíratjuk a képernyőre, hogy leállítottuk a plottolást.

In [ ]:
def start_plot(self):
	self.event_plot.clear()
	t = threading.Thread(target=self.plot_distance_thread)
	t.start()

def stop_plot(self):
	self.event_plot.set()
	print('Measurement is stopped')

#### Az ábrázolás elindítása után az ábra frissítése új adatokkal

Az ábrázolás legfontosabbik része, maga az ábra folytono frissítése, a ```plot_distance_thread``` függvényben történik meg, aminek a bemenő paramétere szintén a ```self```. Nézzük milyen lépések zajlanak le benne. 

Első körben elindítunk egy végtelen ciklust, amit csak a ```self.event_plot``` esemény átállításával lehet leállítani. Ezért a ciklus feltételeként megvizsgáljuk, hogy bekövetkezett-e az átállítás, ```while not self.event_plot.is_set():```. A következő lépésekben frissítjük a legújabb mérésekkel az *x* és *y* tengelyen ábrázolandó adatokat, ```self.line1.set_xdata(self.dlist)```, és ```self.line1.set_ydata(self.ylist)```. Hogy vizuálisan korrektebb legyen, szintén frissítjük az *x* és *y* tengelyek limitjeit, ```self.ax.set_ylim(min(self.ylist)*0.99, max(self.ylist)*1.01)```, ```self.ax.set_xlim(min(self.dlist), max(self.dlist))```. Végül ábrázoljuk a frissített adatokat, ```self.figure.canvas.draw()```, ```self.figure.canvas.flush_events()```. Ahhoz, hogy a mért időpontok szép formázottan jelenjenek meg a tengelyen, meghívjuk a ```plt.gcf().autofmt_xdate()``` függvényt.

Végére pedig beállítjuk, hogy 2 másodpercenként mérjen csak.

In [ ]:
def plot_distance_thread(self):
	while not self.event_plot.is_set():
		self.line1.set_xdata(self.dlist)
		self.line1.set_ydata(self.ylist)
		self.ax.set_ylim(min(self.ylist)*0.99, max(self.ylist)*1.01) # +1 to avoid singular transformation warning
		self.ax.set_xlim(min(self.dlist), max(self.dlist))
		self.figure.canvas.draw()
		self.figure.canvas.flush_events()
		plt.gcf().autofmt_xdate()
		time.sleep(2)

### Az aktív szenzor objektum ábrázolási lehetőséggel

Minden elemet belerakva a kódba a következő programot kapjuk:

```ir_classplotting.py```:

In [ ]:
import matplotlib
matplotlib.use('Qt5Agg')
import threading
import time
from raspberry_functions import read_2column_files, interpolate1d
from gpiozero import LED, MCP3008
import datetime as dt
import matplotlib.pyplot as plt


class ActiveSensor:
	def __init__(self, led, mcp, calibname):
		self.led = led
		self.mcp = mcp
		self.calibfile = calibname
		self.initialize_calibration(self.calibfile)
		self.event = threading.Event()
		self.event_plot = threading.Event()
		self.dlist = []
		self.ylist = []
		
	def start(self):
		self.event.clear()
		self.led.on()

	def start_measurement(self):
		print('Measurement started')
		while not self.event.is_set():
			dd = dt.datetime.now()
			current_voltage = self.mcp.voltage
			current_distance = interpolate1d(self.calib_volt, self.calib_distance, current_voltage)
			print(f'Current distance from object is: {current_distance:.2} cm')
			self.prepare_data(dd, current_distance)
			time.sleep(1)
		
	def stop(self):
		self.event.set()
		self.led.off()
		print('Measurement is stopped')

	def initialize_calibration(self, filename):
		self.calib_volt, self.calib_distance = read_2column_files(filename, header=True)

	def prepare_data(self, dd, yy, maxlen=20):
		self.dlist.append(dd)
		self.ylist.append(yy)
		if len(self.dlist) > maxlen:
			self.dlist.pop(0)
			self.ylist.pop(0)

	def plot_initialize(self):
		plt.ion()
		self.figure, self.ax = plt.subplots(figsize=(8,6))
		self.line1, = self.ax.plot(self.dlist, self.ylist, 'o-')
		plt.title("Dynamic Plot of measurement",fontsize=25)
		plt.xlabel("Time",fontsize=18)
		plt.ylabel("Distance (cm)",fontsize=18)
		plt.grid(True)

	def start_plot(self):
		self.event_plot.clear()
		t = threading.Thread(target=self.plot_distance_thread)
		t.start()

	def stop_plot(self):
		self.event_plot.set()
		print('Measurement is stopped')

	def plot_distance_thread(self):
		while not self.event_plot.is_set():
			self.line1.set_xdata(self.dlist)
			self.line1.set_ydata(self.ylist)
			self.ax.set_ylim(min(self.ylist)*0.99, max(self.ylist)*1.01) # +1 to avoid singular transformation warning
			self.ax.set_xlim(min(self.dlist), max(self.dlist))
			self.figure.canvas.draw()
			self.figure.canvas.flush_events()
			plt.gcf().autofmt_xdate()
			time.sleep(2)


if __name__ == '__main__':
	ir = LED(2)
	mcp = MCP3008(channel=7)
	calib_file = 'ir_calibration.csv'
	a = ActiveSensor(ir, mcp, calib_file)


## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```ir_classplotting.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```ir_classplotting.py```-t elmentettük. Ott begépelve a ```python ir_classplotting.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a terminálból létrehozhatjuk a távolság mérésért felelős objektumot, majd a metódusaival elindíthatjuk és leállíthatjuk a mérést és ezen felül indíthatunk és leállíthatunk plottolást is.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Írd át a klasszt úgy, hogy inicializáláskor bemenő paraméterként megadhasd, szeretnéd-e, hogy kiírja a kód a képernyőre a távolságot vagy sem.
* Írd át a klasszt úgy, hogy inicializáláskor bemenő paraméterként megadhasd, szeretnéd-e, hogy kalibrálja a kód a mért feszültséget távolsággá, vagy sem.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero LED - https://gpiozero.readthedocs.io/en/stable/api_output.html#led

2) MCP3008 datasheet - https://cdn-shop.adafruit.com/datasheets/MCP3008.pdf

3) LM358 datasheet - https://www.ti.com/lit/ds/symlink/lm158-n.pdf , https://components101.com/ics/ic-lm358-pinout-details-datasheet

4) gpiozero MCP3008 - https://gpiozero.readthedocs.io/en/stable/api_spi.html

5) Fájlok kezelése - https://www.programiz.com/python-programming/file-operation

6) Interpolálás - https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html#scipy.interpolate.interp1d

7) threading - https://realpython.com/intro-to-python-threading/

8) Objektum Orientált Programozás (OOP) - https://realpython.com/python3-object-oriented-programming/#:~:text=Programming%20with%20Python.-,What%20Is%20Object%2DOriented%20Programming%20in%20Python%3F,are%20bundled%20into%20individual%20objects.

9) Matplotlib - https://matplotlib.org/stable/tutorials/index.html